In [1]:
import pandas as pd
import numpy as np
import gc
import optuna
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report
import warnings
import joblib
import json

# Suppress warnings
warnings.filterwarnings("ignore")

In [2]:
def load_data_with_scaling_encoding_dtypes():
    # Define dtypes for train_transaction
    dtypes_train_transaction = {
        'TransactionID': 'int32',
        'isFraud': 'int8',
        'TransactionDT': 'int32',
        'TransactionAmt': 'float32',
        'ProductCD': 'object',
        'card1': 'int16',
        'card2': 'float32',
        'card3': 'float32',
        'card4': 'object',
        'card5': 'float32',
        'card6': 'object',
        'addr1': 'float32',
        'addr2': 'float32',
        'dist1': 'float32',
        'dist2': 'float32',
        'P_emaildomain': 'object',
        'R_emaildomain': 'object',
        # Add all C, D, M, V features
        **{f'C{i}': 'float32' for i in range(1, 15)},
        **{f'D{i}': 'float32' for i in range(1, 16)},
        **{f'M{i}': 'object' for i in range(1, 10)},
        **{f'V{i}': 'float32' for i in range(1, 340)}
    }

    # Define dtypes for train_identity
    dtypes_train_identity = {
        **{f'id_{i:02}': 'float32' for i in range(1, 12)},
        'id_12': 'object',
        'id_13': 'float32',
        'id_14': 'float32',
        'id_15': 'object',
        'id_16': 'object',
        **{f'id_{i:02}': 'float32' for i in range(17, 40)},
        **{f'id_{i:02}': 'object' for i in range(23, 39)},
        'DeviceType': 'object',
        'DeviceInfo': 'object'
    }

    # Define dtypes for test datasets
    dtypes_test_transaction = dtypes_train_transaction.copy()
    del dtypes_test_transaction['isFraud']  # Test data does not contain 'isFraud'

    dtypes_test_identity = dtypes_train_identity.copy()

    # Read in datasets with dtypes
    train_transaction = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/train_transaction.csv', 
        dtype=dtypes_train_transaction
    )
    train_identity = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/train_identity.csv', 
        dtype=dtypes_train_identity
    )
    test_transaction = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/test_transaction.csv', 
        dtype=dtypes_test_transaction
    )
    test_identity = pd.read_csv(
        '/kaggle/input/ieee-fraud-detection/test_identity.csv', 
        dtype=dtypes_test_identity
    )

    # Standardize column names
    test_identity.columns = test_identity.columns.str.replace('-', '_')
    test_transaction.columns = test_transaction.columns.str.replace('-', '_')

    # Merge datasets
    train = train_transaction.merge(train_identity, how='left', on='TransactionID')
    test = test_transaction.merge(test_identity, how='left', on='TransactionID')

    # Free up memory
    del train_transaction, train_identity, test_transaction, test_identity
    gc.collect()

    # Handle missing values
    train.fillna(-999, inplace=True)
    test.fillna(-999, inplace=True)

    # Define categorical features
    categorical_features = [
        'ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain',
        'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
        'id_33', 'id_34', 'DeviceType', 'DeviceInfo'
    ]
    categorical_features += [f'id_{i}' for i in range(12, 39)]

    # Encode categorical features
    for col in categorical_features:
        if col in train.columns:
            train[col] = train[col].astype(str)
            test[col] = test[col].astype(str)

            le = LabelEncoder()
            combined_data = pd.concat([train[col], test[col]], axis=0)
            le.fit(combined_data)
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])

    return train, test

# Load data
train, test = load_data_with_scaling_encoding_dtypes()


In [3]:
X = train.drop(columns=['isFraud', 'TransactionID'])
y = train['isFraud']

# Prepare test features
X_test = test.drop(columns=['TransactionID'], errors='ignore')

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [4]:
def optuna_lgb(trial):
    # Dynamically assign GPU
    gpu_id = trial.number % 2  # Alternate between GPU 0 and 1
    print(f"[Trial {trial.number}] Using GPU {gpu_id}")

    # Define LightGBM parameters
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'device': 'gpu',
        'gpu_device_id': gpu_id,  # Assign GPU dynamically
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10)
    }

    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    # Train LightGBM
    model = lgb.train(
        params,
        train_data,
        valid_sets=[val_data],
        num_boost_round=1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(period=100)
        ]
    )

    # Evaluate on validation set
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    auc_score = roc_auc_score(y_val, val_pred)
    print(f"[Trial {trial.number}] Finished with AUC: {auc_score:.4f}")
    return auc_score

# Run Optuna study
study_name = "lightgbm_optimization"
study_lgb = optuna.create_study(study_name=study_name, direction="maximize")
study_lgb.optimize(optuna_lgb, n_trials=50, n_jobs=2)

[I 2025-03-18 16:29:17,948] A new study created in memory with name: lightgbm_optimization


[Trial 0] Using GPU 0
[Trial 1] Using GPU 1
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.907731
[200]	valid_0's auc: 0.918475
[100]	valid_0's auc: 0.929814
[300]	valid_0's auc: 0.926024
[400]	valid_0's auc: 0.932327
[500]	valid_0's auc: 0.937147
[200]	valid_0's auc: 0.943883
[600]	valid_0's auc: 0.94078
[700]	valid_0's auc: 0.943825
[300]	valid_0's auc: 0.951499
[800]	valid_0's auc: 0.946709
[900]	valid_0's auc: 0.948857
[400]	valid_0's auc: 0.956947
[1000]	valid_0's auc: 0.950646
Did not meet early stopping. Best iteration is:
[999]	valid_0's auc: 0.950655


[I 2025-03-18 16:32:23,642] Trial 0 finished with value: 0.9506553615640345 and parameters: {'learning_rate': 0.09123496171341162, 'num_leaves': 71, 'max_depth': 5, 'min_data_in_leaf': 103, 'feature_fraction': 0.6299243860557239, 'bagging_fraction': 0.6962273914565046, 'bagging_freq': 9, 'lambda_l1': 7.203323812499265, 'lambda_l2': 7.083327496261415}. Best is trial 0 with value: 0.9506553615640345.


[Trial 0] Finished with AUC: 0.9507
[Trial 2] Using GPU 0
[500]	valid_0's auc: 0.960403
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.931131
[600]	valid_0's auc: 0.963352
[200]	valid_0's auc: 0.944753
[700]	valid_0's auc: 0.965531
[300]	valid_0's auc: 0.953285
[800]	valid_0's auc: 0.967022
[400]	valid_0's auc: 0.958537
[900]	valid_0's auc: 0.968701
[500]	valid_0's auc: 0.96285
[1000]	valid_0's auc: 0.970001
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.970001


[I 2025-03-18 16:35:26,415] Trial 1 finished with value: 0.9700006458855683 and parameters: {'learning_rate': 0.0443848647948331, 'num_leaves': 137, 'max_depth': 13, 'min_data_in_leaf': 124, 'feature_fraction': 0.6289844352173104, 'bagging_fraction': 0.9341956914045235, 'bagging_freq': 1, 'lambda_l1': 0.15249529357954206, 'lambda_l2': 8.981193789256194}. Best is trial 1 with value: 0.9700006458855683.


...

[Trial 47] Finished with AUC: 0.9732
[Trial 49] Using GPU 1
[200]	valid_0's auc: 0.957415
Training until validation scores don't improve for 100 rounds
[300]	valid_0's auc: 0.963495
[100]	valid_0's auc: 0.942126
[400]	valid_0's auc: 0.966652
[200]	valid_0's auc: 0.955729
[500]	valid_0's auc: 0.969154
[300]	valid_0's auc: 0.961701
[600]	valid_0's auc: 0.970874
[400]	valid_0's auc: 0.965506
[700]	valid_0's auc: 0.972176
[500]	valid_0's auc: 0.967861
[800]	valid_0's auc: 0.973076
[600]	valid_0's auc: 0.969187
[900]	valid_0's auc: 0.973768
[700]	valid_0's auc: 0.970347
[1000]	valid_0's auc: 0.97438
Did not meet early stopping. Best iteration is:
[999]	valid_0's auc: 0.974389


[I 2025-03-18 19:05:44,395] Trial 48 finished with value: 0.9743889420994103 and parameters: {'learning_rate': 0.09989733079450586, 'num_leaves': 97, 'max_depth': 14, 'min_data_in_leaf': 78, 'feature_fraction': 0.6331413277170564, 'bagging_fraction': 0.9824138691508865, 'bagging_freq': 6, 'lambda_l1': 2.5288924114158235, 'lambda_l2': 4.270232714785398}. Best is trial 43 with value: 0.9746674615428745.


[Trial 48] Finished with AUC: 0.9744
[800]	valid_0's auc: 0.971447
[900]	valid_0's auc: 0.971821
[1000]	valid_0's auc: 0.972247
Did not meet early stopping. Best iteration is:
[986]	valid_0's auc: 0.972305


[I 2025-03-18 19:06:30,163] Trial 49 finished with value: 0.9723047346490329 and parameters: {'learning_rate': 0.09990420229603295, 'num_leaves': 95, 'max_depth': 14, 'min_data_in_leaf': 74, 'feature_fraction': 0.6107269020065746, 'bagging_fraction': 0.7459031707025552, 'bagging_freq': 7, 'lambda_l1': 2.5521312175071564, 'lambda_l2': 4.655489449537278}. Best is trial 43 with value: 0.9746674615428745.


[Trial 49] Finished with AUC: 0.9723


In [5]:
best_lgb_params = study_lgb.best_params
with open("lgb_best_params.json", "w") as f:
    json.dump(best_lgb_params, f)

In [6]:
optimized_lgb_model = lgb.train(
    {**best_lgb_params, 'objective': 'binary', 'metric': 'auc'},
    lgb.Dataset(X_train, label=y_train),
    valid_sets=[lgb.Dataset(X_val, label=y_val)],
    num_boost_round=1000,
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=100)
    ]
)


[LightGBM] [Info] Number of positive: 16530, number of negative: 455902
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.897139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 39137
[LightGBM] [Info] Number of data points in the train set: 472432, number of used features: 431
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.034989 -> initscore=-3.317101
[LightGBM] [Info] Start training from score -3.317101
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [7]:
joblib.dump(optimized_lgb_model, "optimized_lgb_model.pkl")

['optimized_lgb_model.pkl']

In [8]:
val_pred = optimized_lgb_model.predict(
    X_val, num_iteration=optimized_lgb_model.best_iteration)
val_predictions = pd.DataFrame(
    {'isFraud': y_val, 'predicted': val_pred})
val_predictions.to_csv("lgb_val_predictions.csv", index=False)

In [9]:
val_auc = roc_auc_score(y_val, val_pred)
print(f"Validation AUC: {val_auc:.4f}")

Validation AUC: 0.9740


In [10]:
test_predictions = optimized_lgb_model.predict(X_test, num_iteration=optimized_lgb_model.best_iteration)

# Save test predictions for submission
submission = pd.DataFrame(
    {'TransactionID': test['TransactionID'],
     'isFraud': test_predictions})
submission.to_csv("submission.csv", index=False)
submission.to_csv("lgb_test_predictions.csv", index=False)

In [11]:
classification_report(y_val,val_pred)

ValueError: Classification metrics can't handle a mix of binary and continuous targets